# Process SmolLM Corpus

In [ ]:
from datasets import load_dataset
import boto3
import gzip,os

# cv2 = load_dataset("HuggingFaceTB/smollm-corpus", "cosmopedia-v2")
# fed = load_dataset("HuggingFaceTB/smollm-corpus", "fineweb-edu-dedup")
# pe = load_dataset("HuggingFaceTB/smollm-corpus", "python-edu")

# os.environ["AWS_SECRET_ACCESS_KEY"]="YOURKEY"
# os.environ["AWS_ACCESS_KEY_ID"]="YOURKEY"

In [ ]:
fed4 = fed.filter(lambda x: x['metadata']['int_score'] >= 4, num_proc=120)
# fed4=load_dataset("chengjunyan1/smollm-10", "fineweb-edu-dedup")

In [ ]:
pe4 = pe.filter(lambda x: x['score'] > 4.16, num_proc=120)
pe4['train'].num_rows,pe4['train'].num_rows/pe['train'].num_rows

In [ ]:
from datasets import dataset_dict

cv2_ratio=(fed4['train'].num_rows/fed['train'].num_rows+pe4['train'].num_rows/pe['train'].num_rows)/2
cv2_samples=int(cv2.num_rows['train']*cv2_ratio)
cv2_10=cv2['train'].shuffle(seed=42).select(range(cv2_samples))
cv2_10=dataset_dict.DatasetDict({'train':cv2_10})

In [ ]:
len_all=fed4['train'].num_rows+pe4['train'].num_rows+cv2_10['train'].num_rows
len_all_full=fed['train'].num_rows+pe['train'].num_rows+cv2['train'].num_rows
print(fed4['train'].num_rows/len_all, pe4['train'].num_rows/len_all, cv2_10['train'].num_rows/len_all)
print(fed['train'].num_rows/len_all_full, pe['train'].num_rows/len_all_full, cv2['train'].num_rows/len_all_full)

In [ ]:
from huggingface_hub import login

login()

In [ ]:
# pe4.push_to_hub("chengjunyan1/smollm-10","python-edu")
# fed4.push_to_hub("chengjunyan1/smollm-10","fineweb-edu-dedup")
# cv2_10.push_to_hub("chengjunyan1/smollm-10","cosmopedia-v2")

# Verify processed corpus

In [ ]:
from datasets import load_dataset,DatasetDict
import boto3
import gzip,os
import functools as ft


# os.environ["AWS_SECRET_ACCESS_KEY"]="YOURKEY"
# os.environ["AWS_ACCESS_KEY_ID"]="YOURKEY"

In [ ]:
fed10=load_dataset("chengjunyan1/smollm-10","fineweb-edu-dedup")
cv10=load_dataset("chengjunyan1/smollm-10","cosmopedia-v2")

In [ ]:
DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download

session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])
s3 = session.client("s3")
bucket_name = "softwareheritage"

def download_contents(blob_id):
    key = f"content/{blob_id}"
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    with gzip.GzipFile(fileobj=obj['Body']) as fin:
        content = fin.read().decode("utf-8", errors="ignore")
    return {"text": content}

def download_python_edu():
    ds = load_dataset("chengjunyan1/smollm-10", "python-edu", split="train", num_proc=DEFAULT_NUM_PROC)
    ds = ds.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
    ds = DatasetDict({"train": ds})
    return ds

pe10=download_python_edu()

In [ ]:
from datasets import concatenate_datasets,DatasetDict
import numpy as np


def resample_dataset(dataset, weight):
    num_samples = int(len(dataset) * weight)
    indices = np.random.choice(len(dataset), num_samples, replace=True)
    return dataset.select(indices)

def combine_datasets(dataset_dicts, weights:dict=None): # weights e.g. {'train':[1.5,1.0]}
    combined_dict = {}
    
    # Initialize weights if not provided
    for dataset_dict in dataset_dicts:
        for key, dataset in dataset_dict.items():
            if key in combined_dict:
                combined_dict[key] = concatenate_datasets([combined_dict[key], dataset])
            else:
                combined_dict[key] = dataset

    # Apply weights by resampling the datasets
    for key in combined_dict:
        datasets = []
        for idx,dataset in enumerate(dataset_dicts):
            if key in dataset:
                if weights is None or key not in weights or weights[key][idx] == 1.0:
                    datasets.append(dataset[key])
                else:
                    resampled_dataset = resample_dataset(dataset[key], weights[key][idx])
                    datasets.append(resampled_dataset)
        combined_dict[key] = concatenate_datasets(datasets)
    
    return DatasetDict(combined_dict)


dataset_dicts=[pe10,fed10,cv10]
combined=combine_datasets(dataset_dicts)

# Create test split based on the processed corpus

In [1]:
import os
from datasets import load_dataset,DatasetDict
import boto3
import gzip,os
from huggingface_hub import login


# login('YOURKEY')

DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download

os.environ["AWS_ACCESS_KEY_ID"]="YOURKEY"
os.environ["AWS_SECRET_ACCESS_KEY"]="YOURKEY"

session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])
s3 = session.client("s3")
bucket_name = "softwareheritage"

def download_contents(blob_id):
    key = f"content/{blob_id}"
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    with gzip.GzipFile(fileobj=obj['Body']) as fin:
        content = fin.read().decode("utf-8", errors="ignore")
    return {"text": content}

def download_python_edu():
    ds = load_dataset("chengjunyan1/smollm-10", "python-edu", split="train", num_proc=DEFAULT_NUM_PROC)
    ds = ds.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
    ds = DatasetDict({"train": ds})
    return ds

In [2]:
pe=load_dataset("HuggingFaceTB/smollm-corpus", "python-edu", num_proc=DEFAULT_NUM_PROC//8)
fed=load_dataset("HuggingFaceTB/smollm-corpus", "fineweb-edu-dedup", num_proc=DEFAULT_NUM_PROC//8)
cv=load_dataset("HuggingFaceTB/smollm-corpus", "cosmopedia-v2", num_proc=DEFAULT_NUM_PROC//8)
# fed10=load_dataset("chengjunyan1/smollm-10","fineweb-edu-dedup",num_proc=DEFAULT_NUM_PROC//8)
# cv10=load_dataset("chengjunyan1/smollm-10","cosmopedia-v2",num_proc=DEFAULT_NUM_PROC//8)
# pe10=download_python_edu()

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/234 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/1934 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/480 [00:00<?, ?it/s]

In [ ]:
# random sample 2 GB of data from each dataset
smollm_size=673
test_ratio=1/smollm_size
pe_test_lines=int(pe['train'].num_rows*test_ratio)
fed_test_lines=int(fed['train'].num_rows*test_ratio)
cv_test_lines=int(cv['train'].num_rows*test_ratio)
pe_shuffle=pe['train'].shuffle(seed=42)
fed_shuffle=fed['train'].shuffle(seed=42)
cv_shuffle=cv['train'].shuffle(seed=42)
pe_test=pe_shuffle.select(range(pe_test_lines))
pe_eval=pe_shuffle.select(range(pe_test_lines,pe_test_lines*2))
pe_test=pe_test.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
pe_eval=pe_eval.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
fed_test=fed_shuffle.select(range(fed_test_lines))
fed_eval=fed_shuffle.select(range(fed_test_lines,fed_test_lines*2))
cv_test=cv_shuffle.select(range(cv_test_lines))
cv_eval=cv_shuffle.select(range(cv_test_lines,cv_test_lines*2))

print('Python-Edu:',pe10.num_rows,pe_eval.num_rows,pe_test.num_rows)
print('FineWeb-Edu-Dedup:',fed10.num_rows,fed_eval.num_rows,fed_test.num_rows)
print('Cosmopedia-V2:',cv10.num_rows,cv_eval.num_rows,cv_test.num_rows)

In [ ]:
pe_test,pe_eval,fed_test,fed_eval,cv_test,cv_eval

In [ ]:
# remove verbatims
import multiprocessing as mp
def get_text(x):
    return x['text']
with mp.Pool(mp.cpu_count()) as pool:
    pe_text=pool.map(get_text, pe_test)+pool.map(get_text, pe_eval)
    fed_text=pool.map(get_text, fed_test)+pool.map(get_text, fed_eval)
    cv_text=pool.map(get_text, cv_test)+pool.map(get_text, cv_eval)

In [ ]:
pe_train=pe10.filter(lambda x: x['text'] not in pe_text, num_proc=DEFAULT_NUM_PROC//4-8)
cv_train=cv10.filter(lambda x: x['text'] not in cv_text, num_proc=DEFAULT_NUM_PROC//4-8)
fed_train=fed10.filter(lambda x: x['text'] not in fed_text, num_proc=DEFAULT_NUM_PROC//4-8)

In [ ]:
print(pe_train['train'].num_rows,pe10['train'].num_rows,pe_train['train'].num_rows/pe10['train'].num_rows)
print(cv_train['train'].num_rows,cv10['train'].num_rows,cv_train['train'].num_rows/cv10['train'].num_rows)
print(fed_train['train'].num_rows,fed10['train'].num_rows,fed_train['train'].num_rows/fed10['train'].num_rows)
total_rows_train=pe_train['train'].num_rows+cv_train['train'].num_rows+fed_train['train'].num_rows
total_rows=pe10['train'].num_rows+cv10['train'].num_rows+fed10['train'].num_rows
print(pe_train['train'].num_rows/total_rows_train,cv_train['train'].num_rows/total_rows_train,fed_train['train'].num_rows/total_rows_train)
print(pe10['train'].num_rows/total_rows,cv10['train'].num_rows/total_rows,fed10['train'].num_rows/total_rows)

In [ ]:
pe10_corpus=DatasetDict({'train':pe10['train'],'eval':pe_eval,'test':pe_test})
fed10_corpus=DatasetDict({'train':fed10['train'],'eval':fed_eval,'test':fed_test})
cv10_corpus=DatasetDict({'train':cv10['train'],'eval':cv_eval,'test':cv_test})

In [ ]:
pe10_corpus,fed10_corpus,cv10_corpus

In [ ]:
# pe10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","python-edu")
# fed10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","fineweb-edu-dedup")
# cv10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","cosmopedia-v2")

In [ ]:
# test the datasets
from datasets import load_dataset
pe10_corpus=load_dataset("chengjunyan1/smollm-10-corpus","python-edu")
fed10_corpus=load_dataset("chengjunyan1/smollm-10-corpus","fineweb-edu-dedup")
cv10_corpus=load_dataset("chengjunyan1/smollm-10-corpus","cosmopedia-v2")

In [ ]:
pe10_corpus,fed10_corpus,cv10_corpus

In [2]:
def download_python_edu(split):
    ds = load_dataset("chengjunyan1/smollm-10-corpus", "python-edu", split=split, num_proc=DEFAULT_NUM_PROC)
    ds = ds.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
    ds = DatasetDict({"train": ds})
    return ds

pe10_train=download_python_edu("train")
pe10_eval=download_python_edu("eval")
pe10_test=download_python_edu("test")
pe10_corpus=DatasetDict({'train':pe10_train['train'],'eval':pe10_eval['train'],'test':pe10_test['train']})

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Parameter 'function'=<function download_contents at 0x7f92d81e19e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map (num_proc=512):   0%|          | 0/864074 [00:00<?, ? examples/s]

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Map (num_proc=512):   0%|          | 0/11409 [00:00<?, ? examples/s]

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Map (num_proc=512):   0%|          | 0/11409 [00:00<?, ? examples/s]

In [10]:
# pe10_corpus,fed10_corpus,cv10_corpus
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')


splits = ['train','eval','test']

# for split in splits:
#     print('Counting',split)
#     fed10_token_lengths=np.sum(fed10_corpus[split]['metadata']['token_count'])
#     print(f'{fed10_token_lengths/1000/1000/1000:.2f}','B')

# for split in splits:
#     print('Counting',split)
#     cv10_token_lengths=np.sum(cv10_corpus[split]['token_length'])
#     print(f'{cv10_token_lengths/1000/1000/1000:.2f}','B')

for split in splits:
    print('Counting',split)
    pe10_length=pe10_corpus[split].map(lambda x: {'token_length':tokenizer(x['text'],return_tensors='pt')['input_ids'].shape[1]},num_proc=cpu_count())
    pe10_token_lengths=np.sum(pe10_length['token_length'])
    print(f'{pe10_token_lengths/1000/1000/1000:.3f}','B')


Counting train
0.513 B
Counting eval


Map (num_proc=128):   0%|          | 0/11409 [00:00<?, ? examples/s]

0.006 B
Counting test


Map (num_proc=128):   0%|          | 0/11409 [00:00<?, ? examples/s]

0.007 B


In [ ]:
#remove text column from pe10

# pe10_corpus['train']=pe10_corpus['train'].remove_columns('text')
# pe10_corpus['eval']=pe10_corpus['eval'].remove_columns('text')
# pe10_corpus['test']=pe10_corpus['test'].remove_columns('text')

# pe10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","python-edu")

# Make train and test splits from scratch

In [ ]:
import os
from datasets import load_dataset,DatasetDict
import boto3
import gzip,os
from huggingface_hub import login
DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download

os.environ["AWS_ACCESS_KEY_ID"]="YOURKEY"
os.environ["AWS_SECRET_ACCESS_KEY"]="YOURKEY"

session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])
s3 = session.client("s3")
bucket_name = "softwareheritage"

def download_contents(blob_id):
    key = f"content/{blob_id}"
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    with gzip.GzipFile(fileobj=obj['Body']) as fin:
        content = fin.read().decode("utf-8", errors="ignore")
    return {"text": content}

def download_python_edu():
    ds = load_dataset("chengjunyan1/smollm-10", "python-edu", split="train", num_proc=DEFAULT_NUM_PROC)
    ds = ds.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
    ds = DatasetDict({"train": ds})
    return ds

In [ ]:
pe=load_dataset("HuggingFaceTB/smollm-corpus", "python-edu", num_proc=DEFAULT_NUM_PROC//8)
fed=load_dataset("HuggingFaceTB/smollm-corpus", "fineweb-edu-dedup", num_proc=DEFAULT_NUM_PROC//8)
cv=load_dataset("HuggingFaceTB/smollm-corpus", "cosmopedia-v2", num_proc=DEFAULT_NUM_PROC//8)

In [ ]:
# random sample 2 GB of data from each dataset
smollm_size=673
test_ratio=1/smollm_size
pe_test_lines=int(pe['train'].num_rows*test_ratio)
fed_test_lines=int(fed['train'].num_rows*test_ratio)
cv_test_lines=int(cv['train'].num_rows*test_ratio)
pe_shuffle=pe['train'].shuffle(seed=42)
fed_shuffle=fed['train'].shuffle(seed=42)
cv_shuffle=cv['train'].shuffle(seed=42)
pe_test=pe_shuffle.select(range(pe_test_lines))
pe_eval=pe_shuffle.select(range(pe_test_lines,pe_test_lines*2))
pe_train=pe_shuffle.select(range(pe_test_lines*2,pe_shuffle.num_rows))
fed_test=fed_shuffle.select(range(fed_test_lines))
fed_eval=fed_shuffle.select(range(fed_test_lines,fed_test_lines*2))
fed_train=fed_shuffle.select(range(fed_test_lines*2,fed_shuffle.num_rows))
cv_test=cv_shuffle.select(range(cv_test_lines))
cv_eval=cv_shuffle.select(range(cv_test_lines,cv_test_lines*2))
cv_train=cv_shuffle.select(range(cv_test_lines*2,cv_shuffle.num_rows))

print('Python-Edu:',pe_train.num_rows,pe_eval.num_rows,pe_test.num_rows)
print('FineWeb-Edu-Dedup:',fed_train.num_rows,fed_eval.num_rows,fed_test.num_rows)
print('Cosmopedia-V2:',cv_train.num_rows,cv_eval.num_rows,cv_test.num_rows)

In [ ]:
pe10 = pe_train.filter(lambda x: x['score'] > 4.16, num_proc=DEFAULT_NUM_PROC//4-8)
fed10 = fed_train.filter(lambda x: x['metadata']['int_score'] >= 4, num_proc=DEFAULT_NUM_PROC//4-8)
cv2_ratio=(fed10['train'].num_rows/fed_train['train'].num_rows+pe10['train'].num_rows/pe_train['train'].num_rows)/2
cv2_samples=int(cv_train.num_rows['train']*cv2_ratio)
cv10=cv_train['train'].shuffle(seed=42).select(range(cv2_samples))
cv10=dataset_dict.DatasetDict({'train':cv10})

In [ ]:
pe10_corpus=DatasetDict({'train':pe10['train'],'eval':pe_eval,'test':pe_test})
fed10_corpus=DatasetDict({'train':fed10['train'],'eval':fed_eval,'test':fed_test})
cv10_corpus=DatasetDict({'train':cv10['train'],'eval':cv_eval,'test':cv_test})

In [ ]:
pe10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","python-edu")
pe10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","fineweb-edu-dedup")
pe10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","cosmopedia-v2")

# V2 Make based on SmolLM Mixture 

In [2]:
import os
from datasets import load_dataset,DatasetDict
import boto3
import gzip,os
from huggingface_hub import login
import numpy as np

# login('YOURKEY')

DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download

os.environ["AWS_ACCESS_KEY_ID"]="YK"
os.environ["AWS_SECRET_ACCESS_KEY"]="YK"

session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])
s3 = session.client("s3")
bucket_name = "softwareheritage"

def download_contents(blob_id):
    key = f"content/{blob_id}"
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    with gzip.GzipFile(fileobj=obj['Body']) as fin:
        content = fin.read().decode("utf-8", errors="ignore")
    return {"text": content}

def download_python_edu(path,split):
    ds = load_dataset(path, "python-edu", split=split, num_proc=DEFAULT_NUM_PROC)
    ds = ds.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
    ds = DatasetDict({split: ds})
    return ds

In [3]:
pe=load_dataset("HuggingFaceTB/smollm-corpus", "python-edu", num_proc=DEFAULT_NUM_PROC//8)
fed=load_dataset("HuggingFaceTB/smollm-corpus", "fineweb-edu-dedup", num_proc=DEFAULT_NUM_PROC//8)
cv=load_dataset("HuggingFaceTB/smollm-corpus", "cosmopedia-v2", num_proc=DEFAULT_NUM_PROC//8)

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/234 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/1934 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/480 [00:00<?, ?it/s]

In [4]:
fed4 = fed.filter(lambda x: x['metadata']['score'] >= 4, num_proc=120)

In [8]:
fed4

DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'metadata'],
        num_rows: 3564328
    })
})

In [12]:
from tqdm import tqdm

fed4_token_lengths=np.sum(fed4['train']['metadata']['token_count'])
# bar=tqdm(fed4['train'])
# for i in bar:
#     fed4_token_lengths+=i['metadata']['token_count']
#     current_count=f'{fed4_token_lengths/1000/1000/1000:.3f}B'
#     bar.set_description(current_count)

0.015B:   0%|          | 14679/3564328 [00:08<33:08, 1785.11it/s]


KeyboardInterrupt: 

## Process DeepMind Math

In [ ]:
from datasets import load_dataset


